# Download annual per capita nightlights 

Create a csv file with annual per capita nightlight values over DHS clusters, for years 2000 to 2013.

Import, authenticate and initialize the earth-engine library

In [1]:
import ee

ee.Authenticate()

ee.Initialize()

Enter verification code: 4/1Adeu5BWsH7hvIeKgW8WGeWk2gDzsOgv3mOXBKDPc57AHALg2STlCgElh_4c

Successfully saved authorization token.


In [2]:
# Import other libraries
import csv
import os
import pandas as pd

Read the csv file with survey points

In [3]:
data_dir = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/interim'
dhs_cluster_file_path = os.path.join(data_dir, 'dhs_treat_control_confounders.csv')
df = pd.read_csv(dhs_cluster_file_path)
unique_countries = df['country'].unique()
df.head()

,country,survey_start_year,year,lat,lon,households,rural,iwi,dhs_id,image_file,...,log_trans_proj_cum_n_2004,log_trans_proj_cum_n_2005,log_trans_proj_cum_n_2006,log_trans_proj_cum_n_2007,log_trans_proj_cum_n_2008,log_trans_proj_cum_n_2009,log_trans_proj_cum_n_2010,log_trans_proj_cum_n_2011,log_trans_proj_cum_n_2012,log_trans_proj_cum_n_2013
0,south_africa,2016,2016,-34.463232,19.542468,6,1,70.723295,48830,./data/dhs_tifs/south_africa_2016/00743.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,south_africa,2016,2016,-34.418873,19.188926,11,0,76.798705,48781,./data/dhs_tifs/south_africa_2016/00694.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,south_africa,2016,2016,-34.412835,19.178965,4,0,81.053723,48828,./data/dhs_tifs/south_africa_2016/00741.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,south_africa,2016,2016,-34.292107,19.563813,6,1,72.766880,48787,./data/dhs_tifs/south_africa_2016/00700.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,south_africa,2016,2016,-34.187500,22.113079,3,0,77.864113,48756,./data/dhs_tifs/south_africa_2016/00669.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Function to create a feature with a 6.7km square around a lat/lon

In [4]:
def createDHSFeature(row):
  lat = ee.Number(row['lat'])
  lon = ee.Number(row['lon'])
  dhs_id = ee.Number(row['dhs_id'])
  properties = {'dhs_id':dhs_id,
               'lat':lat,
               'lon':lon}
    
  #Calc coordinates of a 6.7km bounding box around the point
  feature_radius=3350
  roi = ee.Geometry.Point(lon,lat).buffer(feature_radius).bounds()

  return ee.Feature(roi, properties)

Load nightlight and population density collections over the African continent

In [6]:
africa_region = ee.Geometry.Polygon([
    [-20, 38],   
    [55, 38],    
    [55, -36],   
    [-20, -36]   
])

dmsp = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS').filterBounds(africa_region)
pop_count = ee.ImageCollection(ee.ImageCollection("WorldPop/GP/100m/pop"))

#get a composite image for each nighlight year and store in a dictionary
dmsp_images = {}
for year in range(2000, 2014):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    # Construct the variable name
    variable_name = f"dmsp{year}img"
    img = dmsp.filterDate(start_date, end_date).select(["avg_vis"]).median()
    
    # Create the image using the filterDate and select operations
    dmsp_images[year] = {'img':img,'year':year}

#get an image collection for each population density year and store in a dictionary
pop_count_images = {}
for year in range(2000, 2015):
    if year == 2014:
        break  # Exit the loop when year is 2014, but need it for 2013 processing
        
    start_date = f"{year}-01-01"
    end_date = f"{year+1}-01-01"
    
    # Construct the variable name
    variable_name = f"pop_count{year}img"
    
    # Create the imageCollection using the filterDate operation
    imgCol = pop_count.filterDate(start_date, end_date).select(["population"])
    
    # Store the image using the variable name
    pop_count_images[year] = {'imgCol':imgCol,'year':year}


Function to calculate sum of nightlights within a 6.7km2 square

In [7]:
def calculate_nl_sums(feature):
    return_feature_set = {}   #initialize empty dictionary to return
    
    #calculate annual nighlight sums
    for year, data in dmsp_images.items():
        img = data['img']
        year_value = data['year']

        nlyear = img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            scale=927.67
        )
        sum_nlyear = nlyear.getNumber('avg_vis')
        rounded_nlyear = ee.Number(sum_nlyear).format('%.1f')

        #append this year to the feature set
        prop_name = f'nl{year_value}'
        return_feature_set[prop_name] = rounded_nlyear 
        
    return feature.set(return_feature_set)

Function to calculate sum of population within a 6.7km2 square

In [8]:
def calculate_popcount_sums(feature):
    return_feature_set = {}   #initialize empty dictionary to return
        
    #calculate annual population count sums for this year
    for year, data in pop_count_images.items():
        img_collection = data['imgCol']
        year_value = data['year']     
        
        #Spatially combine country images into one mosaic
        img = img_collection.mosaic()
       
        pcountyear = img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=92.77)

        sum_pcountyear = pcountyear.getNumber('population')
        rounded_sum_pcountyear = ee.Number(sum_pcountyear).format('%.1f')

        #append this year to the feature set
        prop_name = f'pop_count{year_value}'
        return_feature_set[prop_name] = rounded_sum_pcountyear  
    
    return feature.set(return_feature_set)

Get data a country at a time, write to a csv file

In [9]:
# Prepare csv file details
csv_file_path = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/per_cap_nl_dhs_WorldPop.csv'
header = ['dhs_id', 'lat', 'lon', 'nl2000', 'nl2001', 'nl2002',
         'nl2003','nl2004','nl2005','nl2006','nl2007','nl2008',
          'nl2009','nl2010','nl2011','nl2012','nl2013',
          'pop_count2000', 'pop_count2001', 'pop_count2002',
          'pop_count2003','pop_count2004','pop_count2005','pop_count2006','pop_count2007','pop_count2008',
          'pop_count2009','pop_count2010','pop_count2011','pop_count2012','pop_count2013']

# Initialize a list to store the rows for all countries
rows = []

#loop through countries, getting the data and appending to rows list
for country in unique_countries:
    country_df = df[df['country'] == country]
    print(f"Starting: {country}, with n rows: {len(rows)}")

    # create list of features from the country dataframe
    feature_list = [createDHSFeature(row) for index, row in country_df.iterrows()]

    # Create a Feature Collection from the list of features
    country_collection = ee.FeatureCollection(feature_list)

    # calculate nightlights and population count sums
    country_collection2 = country_collection.map(calculate_nl_sums)
    country_collection3 = country_collection2.map(calculate_popcount_sums)

    # Loop through the features in the collection and extract the data
    for feature in country_collection3.getInfo()['features']:
        properties = feature['properties']
        row = [
            properties['dhs_id'],
            properties['lat'],
            properties['lon'],
            properties['nl2000'],
            properties['nl2001'],
            properties['nl2002'],
            properties['nl2003'],
            properties['nl2004'],
            properties['nl2005'],
            properties['nl2006'],
            properties['nl2007'],
            properties['nl2008'],
            properties['nl2009'],
            properties['nl2010'],
            properties['nl2011'],
            properties['nl2012'],
            properties['nl2013'],
            properties['pop_count2000'],
            properties['pop_count2001'],
            properties['pop_count2002'],
            properties['pop_count2003'],
            properties['pop_count2004'],
            properties['pop_count2005'],
            properties['pop_count2006'],
            properties['pop_count2007'],
            properties['pop_count2008'],
            properties['pop_count2009'],
            properties['pop_count2010'],
            properties['pop_count2011'],
            properties['pop_count2012'],
            properties['pop_count2013']    
        ]
        rows.append(row)
        

#Open and write to the csv file; with will close it
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)
    csv_writer.writerows(rows)

print("CSV file has been successfully written.")

Starting: south_africa, with n rows: 0
Starting: lesotho, with n rows: 746
Starting: namibia, with n rows: 1090
Starting: eswatini, with n rows: 1350
Starting: mozambique, with n rows: 1560
Starting: madagascar, with n rows: 2169
Starting: zimbabwe, with n rows: 2429
Starting: zambia, with n rows: 2641
Starting: malawi, with n rows: 2960
Starting: angola, with n rows: 3515
Starting: comoros, with n rows: 3577
Starting: democratic_republic_of_congo, with n rows: 3819
Starting: tanzania, with n rows: 4105
Starting: kenya, with n rows: 4276
Starting: burundi, with n rows: 4665
Starting: gabon, with n rows: 4972
Starting: rwanda, with n rows: 5304
Starting: uganda, with n rows: 5751
Starting: cameroon, with n rows: 5888
Starting: ethiopia, with n rows: 6352
Starting: central_african_republic, with n rows: 6858
Starting: nigeria, with n rows: 6924
Starting: ghana, with n rows: 7280
Starting: ivory_coast, with n rows: 7424
Starting: liberia, with n rows: 7487
Starting: togo, with n rows: 751